In [36]:
import cv2
import numpy as np
import time, random

# Load Yolo
net = cv2.dnn.readNet("weights/yolov3-tiny.weights", "cfg/yolov3-tiny.cfg")
classes = []
with open("data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
# color detection
def detect_color(image):
    return 0

In [48]:
cap = cv2.VideoCapture('dashcamera.mp4')
# cap = cv2.VideoCapture(0)

# get the properties of the input video
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Define the codec and create VideoWriter object to store video
output = cv2.VideoWriter('output.mp4',fourcc, fps, size)

starting_time = time.time()
frame_id = 0
while True:
    ret, frame = cap.read()
    if ret:
        frame_id += 1

        height, width, channels = frame.shape

        

        # Detecting objects
        mean=(0, 0, 0)
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), mean, swapRB=True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # plotting the objects detected
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                if (class_id != 9):
                    continue
                confidence = scores[class_id]
                if confidence > 0.3:
                    # when confidence over threshold object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    # get the target region with traffic lights
                    target_img = frame[y:y+h,x:x+w]
                    # cv2.imwrite('img/target%d.jpg' %frame_id, target_img)

                    # detect color in the region
                    # light_color = detect_color(target_img)
                    light_color = random.choice(['Green', 'Red', 'Yellow'])

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)



        elapsed_time = time.time() - starting_time
        fps = frame_id / elapsed_time
        cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        # cv2.imshow("Image", frame)
        output.write(frame)
        key = cv2.waitKey(1)
        if key == 27:
            break
    else:
        break

cap.release()
# close the videowriter
output.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
